## Постановка задачи
Загрузите данные, приведите их к числовым, заполните пропуски, нормализуйте данные и оптимизируйте память.

Сформируйте параллельный ансамбль (стекинг) из CatBoost, градиентного бустинга, XGBoost и LightGBM. Используйте лучшие гиперпараметры, подобранные ранее, или найдите их через перекрестную проверку. Итоговое решение рассчитайте на основании самого точного предсказания класса у определенной модели ансамбля: выберите для каждого класса модель, которая предсказывает его лучше всего.

Проведите расчеты и выгрузите результат в виде submission.csv

Данные:
* https://video.ittensive.com/machine-learning/prudential/train.csv.gz
* https://video.ittensive.com/machine-learning/prudential/test.csv.gz
* https://video.ittensive.com/machine-learning/prudential/sample_submission.csv.gz

Соревнование: https://www.kaggle.com/c/prudential-life-insurance-assessment/

© ITtensive, 2020

### Подключение библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, confusion_matrix
from catboost import Pool, CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn import preprocessing

### Загрузка данных

In [2]:
data = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/train.csv.gz")
print (data.info()) #Посмотрим сводную информацию методом info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 128 entries, Id to Response
dtypes: float64(18), int64(109), object(1)
memory usage: 58.0+ MB
None


In [3]:
print(data) # прочитаем файл данных, в столбце «Product_Info_2» тип данных смешенный (буква-цифра)

          Id  Product_Info_1 Product_Info_2  Product_Info_3  Product_Info_4  \
0          2               1             D3              10        0.076923   
1          5               1             A1              26        0.076923   
2          6               1             E1              26        0.076923   
3          7               1             D4              10        0.487179   
4          8               1             D2              26        0.230769   
...      ...             ...            ...             ...             ...   
59376  79142               1             D1              10        0.230769   
59377  79143               1             D3              26        0.230769   
59378  79144               1             E1              26        0.076923   
59379  79145               1             D2              10        0.230769   
59380  79146               1             A8              26        0.076923   

       Product_Info_5  Product_Info_6  Product_Info

### Предобработка данных

In [4]:
# Проведем нормализацию данных через предварительную обработку – вызовем функцию  preprocess по параметру «df»
def data_preprocess (df):
#Для предварительной обработки данных переведем смешенное значение параметров 
#"Product_Info_2" отдельно в числовое и буквенное значение     
    df["Product_Info_2_1"] = df["Product_Info_2"].str.slice(0, 1)
    df["Product_Info_2_2"] = pd.to_numeric(df["Product_Info_2"].str.slice(1, 2))
# с помощью функции «to_numeric» преобразуем аргумент в числовой тип
# удалим из набора параметров исходную серию данных, чтобы ее не учитывать в модели.   
    df.drop(labels=["Product_Info_2"], axis=1, inplace=True)
# в массиве данных «Product_Info_2_1» используя метод «unique()» используя уникальные элементы данных
    for l in df["Product_Info_2_1"].unique():
#с помощью функции astype() данные преобразуем  в целочисленный массив «"int8"»
        df["Product_Info_2_1" + l] = df["Product_Info_2_1"].isin([l]).astype("int8")
# удалим из набора параметров  данные «Product_Info_2_1», чтобы ее не учитывать в модели.      
    df.drop(labels=["Product_Info_2_1"], axis=1, inplace=True)
# заменим нулевые значения на «-1»
    df.fillna(value=-1, inplace=True)
    data["Response"] = data["Response"] - 1
    return df # возвращаем значение "df" 

In [5]:
# проведем предварительную обработку данных
data = data_preprocess(data)

### Набор столбцов для расчета

In [6]:
# Выделим набор столбцов для расчета
# проведем группировку данных по столбцам «Страховая история», 
# «Застрахованная информация», «Медицинское ключевое слово», 
# «Семейная история», «История болезни», «Информация о продукте» 
columns_groups = ["Insurance_History", "InsurеdInfo", "Medical_Keyword",
                  "Family_Hist", "Medical_History", "Product_Info"]
# используем данные: рост (Ht), вес (Wt), возраст (Ins_Age), индекс массы тела (BMI),
columns = ["Wt", "Ht", "Ins_Age", "BMI"]
# группируем данные, первый массив данных «columns» добавляет в конец второго массива
for cg in columns_groups:
    columns.extend(data.columns[data.columns.str.startswith(cg)])
print (columns)

['Wt', 'Ht', 'Ins_Age', 'BMI', 'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_5', 'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'Medical_Keyword_1', 'Medical_Keyword_2', 'Medical_Keyword_3', 'Medical_Keyword_4', 'Medical_Keyword_5', 'Medical_Keyword_6', 'Medical_Keyword_7', 'Medical_Keyword_8', 'Medical_Keyword_9', 'Medical_Keyword_10', 'Medical_Keyword_11', 'Medical_Keyword_12', 'Medical_Keyword_13', 'Medical_Keyword_14', 'Medical_Keyword_15', 'Medical_Keyword_16', 'Medical_Keyword_17', 'Medical_Keyword_18', 'Medical_Keyword_19', 'Medical_Keyword_20', 'Medical_Keyword_21', 'Medical_Keyword_22', 'Medical_Keyword_23', 'Medical_Keyword_24', 'Medical_Keyword_25', 'Medical_Keyword_26', 'Medical_Keyword_27', 'Medical_Keyword_28', 'Medical_Keyword_29', 'Medical_Keyword_30', 'Medical_Keyword_31', 'Medical_Keyword_32', 'Medical_Keyword_33', 'Medical_Keyword_34', 'Medical_Keyword_35', 'Medical_Keyword_36', 'M

### Нормализация данных

In [7]:
#Алгоритмы машинного обучения работают лучше или сходятся быстрее, 
#когда данные примерно одинакового масштаба или близки к нормальному 
#распределению.
#Проведем нормализацию данных через предварительную обработку.
#Зададим переменную «scaler» для модуля «preprocessing» функции 
#«StandardScaler()»(стандартный масштабатор), проведем стандартное 
#нормальное распределение (SND) (среднее значение = 0 и данные
#масштабируются до единичной дисперсии).

scaler = preprocessing.StandardScaler()
#Сформируем фрейм данных, с помощью метода «fit_transform» преобразуем его.
data_transformed = pd.DataFrame(scaler.fit_transform(pd.DataFrame(data,
                                                     columns=columns)))
#Сформируем преобразованные столбцы по правилу исходных данных
columns_transformed = data_transformed.columns
#Данные столбца "Response" («Ответ») преобразованных данных соответствуют столбцу исходных данных 
data_transformed["Response"] = data["Response"]

### Оптимизация памяти

In [8]:
'''В целях оптимизации исходных данных максимально уменьшим объем памяти, 
используя функцию оптимизации памяти. Проверим исходный тип данных и 
уменьшим его размер.
Дополнительно переведем буквенные значения в категории. Создадим отдельную 
функцию, которая будет перебирать все серии данных в наборе, для числовых 
серий данных проверять диапазоны значений и автоматически уменьшать 
используемый тип серии данных, если это возможно.  
Отдельно рассмотрим целые и вещественные типы данных.'''
''' вызываем функцию «reduce_mem_usage» по параметру «df», которая перебирает
все существующие столбцы. Суммируем использование столбцов и делим значение 
на 1024², мы получаем использование в МБ. Это помогает значительно сократить 
использование памяти.'''
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        else:
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

In [9]:
# применим функцию «reduce_mem_usage» к фрейму данных «data_transformed»
data_transformed = reduce_mem_usage(data_transformed)
print (data_transformed.info()) #Посмотрим сводную информацию методом info()
# потребление памяти уменьшилось на 75,1 %.

Потребление памяти меньше на 40.49 Мб (минус 75.1 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 119 entries, 0 to Response
dtypes: float16(118), int8(1)
memory usage: 13.4 MB
None


### Построение базовых моделей

In [10]:
# Сформируем фрейм данных Х из преобразованных данных «data_transformed»
x = pd.DataFrame(data_transformed, columns=columns_transformed)

XGBoost

In [11]:
'''Испытаем модель «XGBClassifier»
XGBoost - более регуляризованная форма градиентного бустинга.
Основным преимуществом данной библиотеки является производительность и 
эффективная оптимизация вычислений (лучший результат с меньшей затратой 
ресурсов).
max_depth – максимальная глубина дерева
max_features  – количество признаков, учитываемых алгоритмом для 
построения разветвления  в дереве
n_estimators – число итераций в бустинге.
min_samples_leaf – минимальное число объектов в листе (узле).'''

model_xgb = XGBClassifier(max_depth=17, max_features=27,
                      n_estimators=76, min_samples_leaf=20)
'''С помощью метода  «fit» проведем обучение набора данных «model_xgb» 
на основе фрейма данных Х, исходного набора данных «data», где используем 
только столбец «Response» '''
model_xgb.fit(x, data['Response'])

[19:29:50] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "max_features", "min_samples_leaf" } are not used.



XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=17, max_features=27,
              max_leaves=None, min_child_weight=None, min_samples_leaf=20,
              missing=nan, monotone_constraints=None, n_estimators=76,
              n_jobs=None, num_parallel_tree=None, ...)

CatBoost

In [12]:
'''Испытаем модель «CatBoost»
CatBoost - это библиотека градиентного бустинга, созданная Яндексом. 
Она использует небрежные (oblivious) деревья решений, чтобы вырастить 
сбалансированное дерево. 
Iterations – количество итераций
learning_rate - скорость обучения алгоритма
random_seed - случайное начальное число
depth -  глубина дерева в ансамбле
l2_leaf_reg - коэффициент потерь
loss_function='MultiClass' – метрика обучения «MultiClass»
bootstrap_type - тип начальной загрузки'''
model_cb = CatBoostClassifier(iterations=10000, learning_rate=0.57,
            random_seed=17, depth=6, l2_leaf_reg=2,
            loss_function='MultiClass', bootstrap_type="MVS")
'''С помощью метода  «fit» проведем обучение набора данных «model_cb» на 
основе фрейма данных Х, исходного набора данных «data», где используем 
только столбец «Response» (метка “label”)'''
model_cb.fit(Pool(data=x, label=data["Response"]))

0:	learn: 1.5330128	total: 335ms	remaining: 55m 52s
1:	learn: 1.4359398	total: 487ms	remaining: 40m 35s
2:	learn: 1.3427639	total: 651ms	remaining: 36m 10s
3:	learn: 1.3129551	total: 807ms	remaining: 33m 37s
4:	learn: 1.2920150	total: 1.01s	remaining: 33m 37s
5:	learn: 1.2754757	total: 1.2s	remaining: 33m 16s
6:	learn: 1.2629586	total: 1.35s	remaining: 32m 10s
7:	learn: 1.2538166	total: 1.51s	remaining: 31m 24s
8:	learn: 1.2443132	total: 1.67s	remaining: 30m 49s
9:	learn: 1.2374051	total: 1.81s	remaining: 30m 12s
10:	learn: 1.2335046	total: 1.96s	remaining: 29m 40s
11:	learn: 1.2294222	total: 2.12s	remaining: 29m 25s
12:	learn: 1.2236002	total: 2.27s	remaining: 29m 7s
13:	learn: 1.2143246	total: 2.44s	remaining: 29m 2s
14:	learn: 1.2118135	total: 2.58s	remaining: 28m 39s
15:	learn: 1.2085828	total: 2.74s	remaining: 28m 28s
16:	learn: 1.2040890	total: 2.9s	remaining: 28m 22s
17:	learn: 1.1996522	total: 3.08s	remaining: 28m 25s
18:	learn: 1.1974595	total: 3.24s	remaining: 28m 23s
19:	lea

156:	learn: 0.9832078	total: 25.3s	remaining: 26m 24s
157:	learn: 0.9820599	total: 25.4s	remaining: 26m 23s
158:	learn: 0.9807849	total: 25.6s	remaining: 26m 24s
159:	learn: 0.9798201	total: 25.7s	remaining: 26m 23s
160:	learn: 0.9783707	total: 25.9s	remaining: 26m 23s
161:	learn: 0.9777779	total: 26s	remaining: 26m 21s
162:	learn: 0.9763652	total: 26.2s	remaining: 26m 22s
163:	learn: 0.9762867	total: 26.4s	remaining: 26m 21s
164:	learn: 0.9762719	total: 26.5s	remaining: 26m 18s
165:	learn: 0.9757878	total: 26.6s	remaining: 26m 15s
166:	learn: 0.9749175	total: 26.8s	remaining: 26m 15s
167:	learn: 0.9739074	total: 26.9s	remaining: 26m 14s
168:	learn: 0.9731744	total: 27s	remaining: 26m 13s
169:	learn: 0.9721350	total: 27.2s	remaining: 26m 11s
170:	learn: 0.9713875	total: 27.3s	remaining: 26m 10s
171:	learn: 0.9695542	total: 27.5s	remaining: 26m 9s
172:	learn: 0.9688514	total: 27.6s	remaining: 26m 7s
173:	learn: 0.9679202	total: 27.8s	remaining: 26m 7s
174:	learn: 0.9670706	total: 27.9s	

309:	learn: 0.8613191	total: 48.3s	remaining: 25m 11s
310:	learn: 0.8611482	total: 48.5s	remaining: 25m 10s
311:	learn: 0.8605272	total: 48.6s	remaining: 25m 9s
312:	learn: 0.8595584	total: 48.8s	remaining: 25m 9s
313:	learn: 0.8589036	total: 48.9s	remaining: 25m 8s
314:	learn: 0.8584061	total: 49s	remaining: 25m 7s
315:	learn: 0.8576989	total: 49.2s	remaining: 25m 7s
316:	learn: 0.8569600	total: 49.3s	remaining: 25m 6s
317:	learn: 0.8562195	total: 49.5s	remaining: 25m 6s
318:	learn: 0.8558497	total: 49.6s	remaining: 25m 5s
319:	learn: 0.8553699	total: 49.8s	remaining: 25m 5s
320:	learn: 0.8546782	total: 49.9s	remaining: 25m 4s
321:	learn: 0.8534745	total: 50.1s	remaining: 25m 5s
322:	learn: 0.8524605	total: 50.2s	remaining: 25m 5s
323:	learn: 0.8515460	total: 50.4s	remaining: 25m 4s
324:	learn: 0.8507958	total: 50.5s	remaining: 25m 3s
325:	learn: 0.8502835	total: 50.7s	remaining: 25m 3s
326:	learn: 0.8495768	total: 50.8s	remaining: 25m 3s
327:	learn: 0.8494012	total: 51s	remaining: 25

463:	learn: 0.7630644	total: 1m 11s	remaining: 24m 30s
464:	learn: 0.7621107	total: 1m 11s	remaining: 24m 29s
465:	learn: 0.7616691	total: 1m 11s	remaining: 24m 29s
466:	learn: 0.7609198	total: 1m 11s	remaining: 24m 29s
467:	learn: 0.7605229	total: 1m 12s	remaining: 24m 28s
468:	learn: 0.7603677	total: 1m 12s	remaining: 24m 27s
469:	learn: 0.7599816	total: 1m 12s	remaining: 24m 27s
470:	learn: 0.7590735	total: 1m 12s	remaining: 24m 27s
471:	learn: 0.7587036	total: 1m 12s	remaining: 24m 27s
472:	learn: 0.7579545	total: 1m 12s	remaining: 24m 26s
473:	learn: 0.7572237	total: 1m 12s	remaining: 24m 26s
474:	learn: 0.7565682	total: 1m 13s	remaining: 24m 26s
475:	learn: 0.7558605	total: 1m 13s	remaining: 24m 26s
476:	learn: 0.7554508	total: 1m 13s	remaining: 24m 25s
477:	learn: 0.7548301	total: 1m 13s	remaining: 24m 25s
478:	learn: 0.7540163	total: 1m 13s	remaining: 24m 25s
479:	learn: 0.7535571	total: 1m 13s	remaining: 24m 24s
480:	learn: 0.7525473	total: 1m 14s	remaining: 24m 24s
481:	learn

613:	learn: 0.6880191	total: 1m 33s	remaining: 23m 55s
614:	learn: 0.6880156	total: 1m 34s	remaining: 23m 54s
615:	learn: 0.6878193	total: 1m 34s	remaining: 23m 54s
616:	learn: 0.6875074	total: 1m 34s	remaining: 23m 53s
617:	learn: 0.6871177	total: 1m 34s	remaining: 23m 53s
618:	learn: 0.6862282	total: 1m 34s	remaining: 23m 53s
619:	learn: 0.6855388	total: 1m 34s	remaining: 23m 53s
620:	learn: 0.6852764	total: 1m 34s	remaining: 23m 52s
621:	learn: 0.6846743	total: 1m 35s	remaining: 23m 53s
622:	learn: 0.6841678	total: 1m 35s	remaining: 23m 53s
623:	learn: 0.6839244	total: 1m 35s	remaining: 23m 52s
624:	learn: 0.6830742	total: 1m 35s	remaining: 23m 53s
625:	learn: 0.6824998	total: 1m 35s	remaining: 23m 53s
626:	learn: 0.6822238	total: 1m 35s	remaining: 23m 52s
627:	learn: 0.6822213	total: 1m 35s	remaining: 23m 51s
628:	learn: 0.6811240	total: 1m 36s	remaining: 23m 52s
629:	learn: 0.6803399	total: 1m 36s	remaining: 23m 52s
630:	learn: 0.6798987	total: 1m 36s	remaining: 23m 52s
631:	learn

...

9340:	learn: 0.0261695	total: 23m 47s	remaining: 1m 40s
9341:	learn: 0.0261628	total: 23m 47s	remaining: 1m 40s
9342:	learn: 0.0261588	total: 23m 47s	remaining: 1m 40s
9343:	learn: 0.0261551	total: 23m 47s	remaining: 1m 40s
9344:	learn: 0.0261501	total: 23m 47s	remaining: 1m 40s
9345:	learn: 0.0261494	total: 23m 48s	remaining: 1m 39s
9346:	learn: 0.0261494	total: 23m 48s	remaining: 1m 39s
9347:	learn: 0.0261435	total: 23m 48s	remaining: 1m 39s
9348:	learn: 0.0261406	total: 23m 48s	remaining: 1m 39s
9349:	learn: 0.0261366	total: 23m 48s	remaining: 1m 39s
9350:	learn: 0.0261232	total: 23m 48s	remaining: 1m 39s
9351:	learn: 0.0261191	total: 23m 49s	remaining: 1m 39s
9352:	learn: 0.0261123	total: 23m 49s	remaining: 1m 38s
9353:	learn: 0.0261046	total: 23m 49s	remaining: 1m 38s
9354:	learn: 0.0260978	total: 23m 49s	remaining: 1m 38s
9355:	learn: 0.0260935	total: 23m 49s	remaining: 1m 38s
9356:	learn: 0.0260841	total: 23m 49s	remaining: 1m 38s
9357:	learn: 0.0260815	total: 23m 50s	remaining:

9489:	learn: 0.0254801	total: 24m 10s	remaining: 1m 17s
9490:	learn: 0.0254799	total: 24m 10s	remaining: 1m 17s
9491:	learn: 0.0254741	total: 24m 10s	remaining: 1m 17s
9492:	learn: 0.0254710	total: 24m 10s	remaining: 1m 17s
9493:	learn: 0.0254662	total: 24m 10s	remaining: 1m 17s
9494:	learn: 0.0254638	total: 24m 10s	remaining: 1m 17s
9495:	learn: 0.0254638	total: 24m 10s	remaining: 1m 17s
9496:	learn: 0.0254536	total: 24m 11s	remaining: 1m 16s
9497:	learn: 0.0254467	total: 24m 11s	remaining: 1m 16s
9498:	learn: 0.0254449	total: 24m 11s	remaining: 1m 16s
9499:	learn: 0.0254344	total: 24m 11s	remaining: 1m 16s
9500:	learn: 0.0254331	total: 24m 11s	remaining: 1m 16s
9501:	learn: 0.0254242	total: 24m 11s	remaining: 1m 16s
9502:	learn: 0.0254083	total: 24m 12s	remaining: 1m 15s
9503:	learn: 0.0254049	total: 24m 12s	remaining: 1m 15s
9504:	learn: 0.0253990	total: 24m 12s	remaining: 1m 15s
9505:	learn: 0.0253969	total: 24m 12s	remaining: 1m 15s
9506:	learn: 0.0253960	total: 24m 12s	remaining:

9639:	learn: 0.0248543	total: 24m 33s	remaining: 55s
9640:	learn: 0.0248495	total: 24m 33s	remaining: 54.9s
9641:	learn: 0.0248481	total: 24m 33s	remaining: 54.7s
9642:	learn: 0.0248460	total: 24m 33s	remaining: 54.5s
9643:	learn: 0.0248460	total: 24m 33s	remaining: 54.4s
9644:	learn: 0.0248441	total: 24m 33s	remaining: 54.2s
9645:	learn: 0.0248341	total: 24m 33s	remaining: 54.1s
9646:	learn: 0.0248225	total: 24m 34s	remaining: 53.9s
9647:	learn: 0.0248110	total: 24m 34s	remaining: 53.8s
9648:	learn: 0.0247964	total: 24m 34s	remaining: 53.6s
9649:	learn: 0.0247940	total: 24m 34s	remaining: 53.5s
9650:	learn: 0.0247855	total: 24m 34s	remaining: 53.3s
9651:	learn: 0.0247821	total: 24m 34s	remaining: 53.2s
9652:	learn: 0.0247697	total: 24m 35s	remaining: 53s
9653:	learn: 0.0247510	total: 24m 35s	remaining: 52.9s
9654:	learn: 0.0247444	total: 24m 35s	remaining: 52.7s
9655:	learn: 0.0247406	total: 24m 35s	remaining: 52.6s
9656:	learn: 0.0247389	total: 24m 35s	remaining: 52.4s
9657:	learn: 0

9790:	learn: 0.0240068	total: 24m 56s	remaining: 32s
9791:	learn: 0.0240064	total: 24m 57s	remaining: 31.8s
9792:	learn: 0.0240030	total: 24m 57s	remaining: 31.6s
9793:	learn: 0.0239978	total: 24m 57s	remaining: 31.5s
9794:	learn: 0.0239939	total: 24m 57s	remaining: 31.3s
9795:	learn: 0.0239921	total: 24m 57s	remaining: 31.2s
9796:	learn: 0.0239887	total: 24m 57s	remaining: 31s
9797:	learn: 0.0239799	total: 24m 57s	remaining: 30.9s
9798:	learn: 0.0239779	total: 24m 58s	remaining: 30.7s
9799:	learn: 0.0239758	total: 24m 58s	remaining: 30.6s
9800:	learn: 0.0239754	total: 24m 58s	remaining: 30.4s
9801:	learn: 0.0239672	total: 24m 58s	remaining: 30.3s
9802:	learn: 0.0239609	total: 24m 58s	remaining: 30.1s
9803:	learn: 0.0239533	total: 24m 58s	remaining: 30s
9804:	learn: 0.0239446	total: 24m 59s	remaining: 29.8s
9805:	learn: 0.0239425	total: 24m 59s	remaining: 29.7s
9806:	learn: 0.0239298	total: 24m 59s	remaining: 29.5s
9807:	learn: 0.0239256	total: 24m 59s	remaining: 29.4s
9808:	learn: 0.0

9942:	learn: 0.0233525	total: 25m 20s	remaining: 8.72s
9943:	learn: 0.0233462	total: 25m 20s	remaining: 8.56s
9944:	learn: 0.0233388	total: 25m 20s	remaining: 8.41s
9945:	learn: 0.0233328	total: 25m 20s	remaining: 8.26s
9946:	learn: 0.0233313	total: 25m 21s	remaining: 8.1s
9947:	learn: 0.0233298	total: 25m 21s	remaining: 7.95s
9948:	learn: 0.0233297	total: 25m 21s	remaining: 7.8s
9949:	learn: 0.0233266	total: 25m 21s	remaining: 7.64s
9950:	learn: 0.0233266	total: 25m 21s	remaining: 7.49s
9951:	learn: 0.0233264	total: 25m 21s	remaining: 7.34s
9952:	learn: 0.0233247	total: 25m 21s	remaining: 7.19s
9953:	learn: 0.0233212	total: 25m 22s	remaining: 7.03s
9954:	learn: 0.0233100	total: 25m 22s	remaining: 6.88s
9955:	learn: 0.0232954	total: 25m 22s	remaining: 6.73s
9956:	learn: 0.0232928	total: 25m 22s	remaining: 6.58s
9957:	learn: 0.0232895	total: 25m 22s	remaining: 6.42s
9958:	learn: 0.0232843	total: 25m 22s	remaining: 6.27s
9959:	learn: 0.0232799	total: 25m 23s	remaining: 6.12s
9960:	learn:

Градиентный бустинг

In [13]:
print (x.info()) #Посмотрим сводную информацию методом info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 118 entries, 0 to 117
dtypes: float16(118)
memory usage: 13.4 MB
None


In [14]:
'''Деревья для градиентного бустинга строятся последовательно для 
минимизации ошибки предыдущего дерева (или деревьев). При этом в самом 
дереве разбиение выполняется по минимизации информационных потерь, 
без учета сортировки исходных данных по количеству информации в них.

Построим ансамбль решающих деревьев, используя градиентный бустинг 
(GradientBoostingClassifier). 
Используем параметры: 
random_state – начальное значение
max_depth – максимальная глубина дерева
max_features - максимальное количество признаков, учитываемых алгоритмом 
для построения разветвления  в дереве
min_samples_leaf - минимальное количество выборок
n_estimators - количество оценщиков
'''
model_gbc = GradientBoostingClassifier(random_state=17, max_depth=13,
                max_features=26, min_samples_leaf=21, n_estimators=75)
'''С помощью метода  «fit» проведем обучение набора данных «model_gbc» на 
основе фрейма данных Х, исходного набора данных «data», где используем 
только столбец «Response» '''
model_gbc.fit(x, data['Response'])

GradientBoostingClassifier(max_depth=13, max_features=26, min_samples_leaf=21,
                           n_estimators=75, random_state=17)

LightGBM

In [15]:
'''LightGBM - облегченный  градиентный бустинг
Основное отличие этого градиентного бустинга от предыдущих - использование 
сильно-разнородных (определяется разностью, гистограммой самих данных) 
экземпляров в выборке для формирования первоначального дерева: сначала 
рассматриваются все крайние, "плохие", случаи, а затем к ним "достраиваются" 
средние, "хорошие". Это позволяет еще быстрее минимизировать ошибку моделей.
Из дополнительных плюсов: алгоритм запускается сразу на всех ядрах процессора, 
max_depth – максимальная глубина дерева
min_child_samples - минимальное число элементов выборке в листе
num_leaves - число листьев в каждом дереве
n_estimators - количество оценщиков'''
model_lgb = lgb.LGBMRegressor(random_state=17, max_depth=18,
    min_child_samples=17, num_leaves=35, n_estimators=10000)
'''С помощью метода  «fit» проведем обучение набора данных «model_lgb» на 
основе фрейма данных Х, исходного набора данных «data», где используем
только столбец «Response» '''
model_lgb.fit(x, data['Response'])

LGBMRegressor(max_depth=18, min_child_samples=17, n_estimators=10000,
              num_leaves=35, random_state=17)

### Загрузка данных для расчета

In [16]:
data_test = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/test.csv.gz")
#Применим функцию «preprocess» к массиву  данных «data_test», необходимых для дальнейших расчетов
data_test = data_preprocess(data_test) 
#С помощью функции « reduce_mem_usage»  уменьшим пространство, занимаемое данными в памяти, путем настройки типа данных
data_test = reduce_mem_usage(data_test)
#Сформируем фрейм данных, с помощью метода «transform» преобразуем его.
data_test_transformed = pd.DataFrame(scaler.transform(pd.DataFrame(data_test,
                                                     columns=columns)))
print (data_test_transformed.info()) #Посмотрим сводную информацию методом info()

Потребление памяти меньше на 16.34 Мб (минус 84.9 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19765 entries, 0 to 19764
Columns: 118 entries, 0 to 117
dtypes: float32(118)
memory usage: 8.9 MB
None


### Расчет предказаний
Проведем расчет предсказаний для четырех моделей

In [17]:
data_test["target_xgb"] = model_xgb.predict(data_test_transformed)

C:\Users\Mikhail\AppData\Local\Temp\ipykernel_6172\1008409102.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_test["target_xgb"] = model_xgb.predict(data_test_transformed)


In [18]:
data_test["target_cb"] = model_cb.predict(Pool(data=data_test_transformed))

C:\Users\Mikhail\AppData\Local\Temp\ipykernel_6172\1488459043.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_test["target_cb"] = model_cb.predict(Pool(data=data_test_transformed))


In [19]:
data_test["target_gbc"] = model_gbc.predict(data_test_transformed)

C:\Users\Mikhail\AppData\Local\Temp\ipykernel_6172\1559723127.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_test["target_gbc"] = model_gbc.predict(data_test_transformed)


In [20]:
data_test["target_lgb"] = np.round(model_lgb.predict(data_test_transformed)).astype("int8")

C:\Users\Mikhail\AppData\Local\Temp\ipykernel_6172\2769997431.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_test["target_lgb"] = np.round(model_lgb.predict(data_test_transformed)).astype("int8")


In [24]:
### Посмотрим предсказания для четырех моделей


In [25]:
print(data_test["target_xgb"])

0        6
1        7
2        5
3        6
4        7
        ..
19760    7
19761    6
19762    5
19763    1
19764    3
Name: target_xgb, Length: 19765, dtype: int64


In [26]:
print(data_test["target_cb"])

0        1
1        7
2        5
3        0
4        7
        ..
19760    7
19761    7
19762    0
19763    1
19764    3
Name: target_cb, Length: 19765, dtype: int64


In [27]:
print(data_test["target_gbc"])

0        0
1        7
2        5
3        6
4        7
        ..
19760    7
19761    7
19762    0
19763    1
19764    0
Name: target_gbc, Length: 19765, dtype: int64


In [28]:
print(data_test["target_lgb"])

0        4
1        5
2        4
3        6
4        6
        ..
19760    7
19761    7
19762    1
19763    2
19764    2
Name: target_lgb, Length: 19765, dtype: int8


Классы смещены на 1: начинаются от 0 и заканчиваются 7. Судя по рассчитанным матрицам ошибок, для 0, 1, 3, 4 и 6 классов точнее работает градиентный бустинг, для 2 - XGBoost, для 5 - LightGBM, для 7 - логистическая регрессия.

Точные параметры классов можно перерассчитать, например, через перекрестную проверку всех данных.

In [29]:
# Вызовим функцию "vote_class" по параметру "x", уточним параметры классов для моделей
def vote_class (x):
    if x.target_xgb == 2:
        class_ = x.target_xgb
#    elif x.target_lgb == 7:
#        class_ = x.target_lgb
#    elif x.target_cb == 0:
#        class_ = x.target_cb
    else:
        class_ = x.target_gbc
    x["Response"] = class_ + 1
    return x #возвращаем значение "x"  

In [30]:
# преобразуем массив данных "data_test" по условиям функции "vote_class"
data_test = data_test.apply(vote_class, axis=1)
# просмотрим первые пять строк в Dataframe «data_test»
print (data_test.head())

     Id  Product_Info_1  Product_Info_3  Product_Info_4  Product_Info_5  \
0   1.0             1.0            26.0        0.487061             2.0   
1   3.0             1.0            26.0        0.076904             2.0   
2   4.0             1.0            26.0        0.144653             2.0   
3   9.0             1.0            26.0        0.151733             2.0   
4  12.0             1.0            26.0        0.076904             2.0   

   Product_Info_6  Product_Info_7   Ins_Age        Ht        Wt  ...  \
0             3.0             1.0  0.611816  0.781738  0.338867  ...   
1             3.0             1.0  0.626953  0.727051  0.311768  ...   
2             3.0             1.0  0.582031  0.708984  0.320068  ...   
3             1.0             1.0  0.522461  0.654785  0.267822  ...   
4             3.0             1.0  0.298584  0.672852  0.246826  ...   

   Product_Info_2_1D  Product_Info_2_1A  Product_Info_2_1E  Product_Info_2_1B  \
0                1.0               

### Формирование и выгрузка результатов
Загрузим примерный файл, заменим в нем результаты и сохраним.

Число строк в файле будет равно размену набора данных + 1 заголовочная строка.

In [31]:
submission = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/sample_submission.csv.gz")
submission["Response"] = data_test["Response"].astype("int8")
submission.to_csv("submission.csv", index=False)
print (len(submission["Response"]) + 1)

19766


### Само-проверка модели
Рассчитаем точность классификации на обучающих данных

In [32]:
data_copy = data_transformed.copy()
x_copy = pd.DataFrame(data_copy, columns=columns_transformed)
copy_dataset = Pool(data=x_copy, label=data_copy["Response"])
data_copy["target_xgb"] = model_xgb.predict(x_copy)
data_copy["target_cb"] = model_cb.predict(copy_dataset)
data_copy["target_gbc"] = model_gbc.predict(x_copy)
data_copy["target_lgb"] = np.round(model_lgb.predict(x_copy)).astype("int8")

In [33]:
class_target = ["target_gbc"]*8
def vote_class_enumerate (x):
    for _,target in enumerate(class_target):
        if x[target] == _:
            x["Response"] = x[target]
            break
    return x

In [34]:
kappa_min = 0
for target_model in ["xgb", "cb", "gbc", "lgb"]:
    print ("Проверяем модель:", target_model)
    target_model = "target_" + target_model
    for class_ in range(0,8):
        target_model_prev = class_target[class_]
        class_target[class_] = target_model
        data_copy = data_copy.apply(vote_class_enumerate, axis=1)
        kappa = cohen_kappa_score(data_copy["Response"], 
                data["Response"], weights='quadratic')
        if kappa > kappa_min:
            kappa_min = kappa
        else:
            class_target[class_] = target_model_prev
    print ("Максимальная оценка:", kappa_min)
print (class_target)

Проверяем модель: xgb
Максимальная оценка: 0.91923523748714
Проверяем модель: cb
Максимальная оценка: 0.9234997580080131
Проверяем модель: gbc
Максимальная оценка: 0.9235370466509727
Проверяем модель: lgb
Максимальная оценка: 0.9239036848645986
['target_cb', 'target_cb', 'target_xgb', 'target_xgb', 'target_gbc', 'target_cb', 'target_lgb', 'target_xgb']


In [35]:
data_copy = data_copy.apply(vote_class_enumerate, axis=1)

In [36]:
print ("Результат:",
       round(cohen_kappa_score(data_copy["Response"],
                    data["Response"], weights='quadratic'), 3))
print (confusion_matrix(data_copy["Response"], data["Response"]))

Результат: 0.924
[[    0     0     0     0     0     0     0     0     0]
 [ 6207     0     0     0     0     0     0     0     0]
 [    0  6552     0     0     0     0     0     0     0]
 [    0     0  1013     0     0     0     0     0     0]
 [    0     0     0  1428     7     0     0     0     0]
 [    0     0     0     0  5306    84    13    10     0]
 [    0     0     0     0   118 11149   202     0     0]
 [    0     0     0     0     1     0  7793   862     0]
 [    0     0     0     0     0     0    19 18617     0]]
